<b> Testing Open3D for Depth rendering </b>

In [ ]:
import open3d as o3d
import cv2
from PIL import Image
import numpy as np
import torch
import os
import matplotlib.pyplot as plt

### <b> Point Cloud (.ply) to 2D top down image (.png) </b>

In [ ]:
#point_cloud = o3d.io.read_point_cloud("../../ply/raw/flexx2_pointcloud_50GA1_2024-09-12T18_23_11Z.ply")
#o3d.visualization.draw_geometries([point_cloud])

In [ ]:
'''
print(f"Number of points: {len(point_cloud.points)}")
print(f"Has colors: {point_cloud.has_colors()}")
print(f"Has normals: {point_cloud.has_normals()}")


o3d.visualization.draw_geometries(
    [point_cloud],
    window_name="Point Cloud Visualization",
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=True 
)
'''

In [ ]:
def pointcloud_to_image_cv2(pcd, width=1456, height=1092):
    """
    Convert point cloud file to top-down 2D depth image
    """
    points = np.asarray(pcd.points)
    
    # Check if point cloud is empty
    if len(points) == 0:
        return np.zeros((height, width), dtype=np.float32)
    
    # Get bounds
    x_min, x_max = points[:, 0].min(), points[:, 0].max()
    y_min, y_max = points[:, 1].min(), points[:, 1].max()
    
    # Check for degenerate bounds
    if x_min == x_max or y_min == y_max:
        return np.zeros((height, width), dtype=np.float32)
    
    # Convert to pixel coordinates
    u = ((points[:, 0] - x_min) / (x_max - x_min) * (width - 1)).astype(int)
    v = ((points[:, 1] - y_min) / (y_max - y_min) * (height - 1)).astype(int)
    
    # Always create single-channel depth image
    image = np.zeros((height, width), dtype=np.float32)
    for i in range(len(u)):
        cv2.circle(image, (u[i], v[i]), 1, float(points[i, 2]), -1)
    
    return image

In [ ]:
#point_cloud = o3d.io.read_point_cloud("../../ply/raw/flexx2_pointcloud_75GA2_2024-09-20T8_52_12Z.ply")
#o3d.visualization.draw_geometries([point_cloud])
#im_75GA2 = pointcloud_to_image_cv2(point_cloud)
#cv2.imwrite("pointcloud_topdown.png", im_75GA2)

In [ ]:
point_clouds_dir = "../../ply/raw/"

In [ ]:
def convert_all_ply(point_cloud_directory):

    point_clouds = os.listdir(point_cloud_directory)
    depth_png_directory = "../../Depth_INESC/"
    print(f"Total number of point clouds {len(point_clouds)}\n")
    
    if not os.path.exists(depth_png_directory):
        os.makedirs(depth_png_directory)

    for idx, p_cloud in enumerate(point_clouds):
        
        new_file_name = p_cloud[18:34] + '.png'
        save_here = depth_png_directory + new_file_name
        
        point_cloud = o3d.io.read_point_cloud(point_cloud_directory + p_cloud)
        two_d_png = pointcloud_to_image_cv2(point_cloud)
        
        # Convert depth to millimeters and save as 16-bit (preserve spatial information)
        depth_mm = (two_d_png * 1000).astype(np.uint16)
        cv2.imwrite(save_here, depth_mm)
        
        print(f"Point cloud {idx}/{len(point_clouds)-1} converted")

    print(f"2D top-down depth visualizations created and saved at {depth_png_directory}.")

In [ ]:
convert_all_ply(point_cloud_directory=point_clouds_dir)

In [ ]:
depth_png_directory = "../../Depth_INESC/"
print(f"Directory exists: {os.path.exists(depth_png_directory)}")
print(f"Full path: {os.path.abspath(depth_png_directory)}")

In [ ]:
#rgb_image = Image.open("../../NewRGBImages/images_with_gt/50GA1_2024-09-07.png")
#rgb_width, rgb_height = rgb_image.size
#print(f"Width: {rgb_width} px | Height: {rgb_height} px")

In [ ]:
depth_path = "../../Depth_INESC/50GA1_2024-09-12.png"
depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
depth_tensor = torch.from_numpy(depth).float()  # (H, W)
depth_tensor = depth_tensor.unsqueeze(0)
print(type(depth_tensor))
print(f"Max. value: {torch.max(depth_tensor)}| Min. value: {torch.min(depth_tensor)}")


In [ ]:
depth_tensor.shape

### <b> Check distance to camera distribution for depth images </b>

In [ ]:
def depth_values_frequency_histogram(depth_path):

    images = [f for f in os.listdir(depth_path)]
    print(f"Images: {images}")
    num_images = len(images)
    print(num_images)
    height, width = 1092, 1456 # CHECK IF THIS IS CORRECT (or the other way around)
    all_values = np.zeros((num_images * height * width,), dtype=np.uint16)

    print(f"Preallocated shape: {all_values.shape}")

    idx = 0 # For smart allocation
    for i,img in enumerate(images):
        img_path = os.path.join(depth_path, img)

        # Read raw depth image with original bit-depth (likely uint16)
        depth_raw = cv2.imread(depth_path+'/'+img, cv2.IMREAD_UNCHANGED)

        if depth_raw is None:
            print(f"Warning: Failed to load {depth_path+'/'+img}")
            continue

        print(f"({i}/{num_images-1}) Reading {img_path} | dtype: {depth_raw.dtype} | shape: {depth_raw.shape}")

        # Flatten and append raw values
        flat = depth_raw.flatten()
        all_values[idx:idx + flat.size] = flat
        idx += flat.size
    
    # Plot histogram
    plt.hist(all_values, bins=100)
    plt.title("Raw Depth Value Distribution (All Images)")
    plt.xlabel("Depth Value")
    plt.ylabel("Count")
    plt.show()

In [ ]:
depth_values_frequency_histogram(depth_path= "../../Depth_INESC/")

#### <b> O que preciso:</b>

* Sistema de coordenadas: Assegura-me de que a point cloud está no mesmo sistema de coordenadas do que a câmara
* Calibração da câmara: Usar os parâmetros de calibração da câmara (comprimento focal x e y, altura a que a câmara estava durante a experiência) -> Para projeções 3D p/ 2D: <b> DATA SHEET: através da dist focal e souber a área super da alface e comparar com lidar 3d obtenho o fator de escala p/ conversão do sist de coordenadas (consigo inferir altura) </b>
* Escala de Profundidate/Depth: Qual é a escala em milímetros? <b> METROS </b>
* Transformação de sistema de coordenadas LiDAR -> Câmara